In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras import regularizers
from PIL import Image

def create_model(input_shape, dense_size, classes):
    x = Input(shape=(input_shape))
    y = Flatten()(x)
    y = Dense(classes, activation='softmax', name='dense_layer')(y)
    model = Model(inputs=x, outputs=y)
    return model

def load_labels(filename):
    with open(filename,'r') as file:
       li = file.readlines()
    label_count = len(li)
    labels = np.empty((label_count,1), dtype='int')
    i = 0
    with open(filename) as f:
        for line in f:
            labels[i] = int(line.replace("\n", ""))
            i = i + 1
    return labels

def load_images(folder, image_count, image_size):
    array_shape = (image_count, image_size[0], image_size[1], image_size[2])
    imageset = np.empty(array_shape, dtype='float')
    for i in range(0,image_count):
        image = Image.open(folder + '/image_' + "{:04d}".format(i) + '.png')
        imageset[i] = np.asarray(image)
    return imageset

def normalize_dataset(sampled_images):
	sampled_images = (sampled_images.astype('float32')-128) / 128
	return sampled_images

def split_test_val(data, splitpoint):
    return data[splitpoint:], data[:splitpoint]

########################################################################
Y_test = load_labels('testing/labels.csv')
X_test = load_images('testing', len(Y_test), (32,32,3))

y_train = load_labels('training/labels.csv')
x_train = load_images('training', len(y_train), (32,32,3))

x_train = normalize_dataset(x_train)

##### Test set preparation #####
splitpoint = 2000
X_test = normalize_dataset(X_test)
x_test, x_val = split_test_val(X_test, splitpoint)
y_test, y_val = split_test_val(Y_test, splitpoint)
print('Validation set size', x_val.shape)
print('Test set size:', x_test.shape)
print('')

class_count = len(np.unique(y_train))

# define a neural network for training
dims = (32, 32, 3)
dense_sz = 100
model = create_model(dims, dense_sz, class_count)

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


c:\Users\danie\Documents\Projects\ml4cv\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


FileNotFoundError: [Errno 2] No such file or directory: 'testing/labels.csv'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras import regularizers
from PIL import Image

# 1. DATA LOADING AND NORMALIZATION
def load_images(folder, image_count, image_size):
    array_shape = (image_count, image_size[0], image_size[1], image_size[2])
    imageset = np.empty(array_shape, dtype='float')
    for i in range(0, image_count):
        # Open image and convert to numpy array
        image = Image.open(f"{folder}/image_{i:04d}.png")
        # DATA NORMALIZATION: Mapping [0, 255] to [-1, 1]
        # Formula: (original - 128) / 128
        imageset[i] = (np.asarray(image).astype('float') - 128.0) / 128.0
    return imageset

# 2. MODEL ARCHITECTURE (Linear / Dense Network)
def create_model(input_shape, dense_size, classes, reg_strength=0.0):
    x = Input(shape=input_shape)
    
    # Flatten converts (32, 32, 3) image to a 1D vector of 3072 values
    y = Flatten()(x)
    
    # Hidden Layer with ReLU activation and L2 Regularization
    # L2 helps prevent overfitting by penalizing large weights
    y = Dense(dense_size, activation='relu', 
              kernel_regularizer=regularizers.l2(reg_strength))(y)
    
    # Output Layer: Softmax turns scores into probabilities for classification
    y = Dense(classes, activation='softmax', name='output_layer')(y)
    
    model = Model(inputs=x, outputs=y)
    return model

# 3. TRAINING CONFIGURATION
model = create_model(input_shape=(32, 32, 3), dense_size=512, classes=10, reg_strength=0.03)

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. WEIGHT VISUALIZATION (Main concept for the exam)
# This snippet shows how to "see" what a neuron has learned
weights, biases = model.get_layer('output_layer').get_weights()
# Shape is (inputs, outputs) -> (3072, 10)
# To visualize, we reshape the weights back to (32, 32, 3)
weight_image = weights[:, 0].reshape(32, 32, 3)